# Introduction

- Cet ensemble de notebooks présente l'utilisation de réseaux neuronaux de plus en plus complexes pour résoudre un problème de reconnaissance de chiffres manuscrits avec la plus grande précision possible.
- Ce travail se base sur l'excellente vidéo de Martin Gorner [TensorFlow and Deep Learning without a PhD](www.youtube.com/watch?v=u4alGiomYP4) et les ressources correspondantes sur [Github](https://github.com/GoogleCloudPlatform/tensorflow-without-a-phd)
- Le but est d'en donner une version en français sous jupyter notebook qui détaille tous les pas de la construction d'une solution efficace

Dans ce premier document nous étudierons l'utilisation d'un réseau de neurones simple et les fondements mathématiques qui lui sont liés.

# Première version du programme: Softmax et Cross Entropy

- Le but de ce programme est de reconnaître des chiffres manuscrits issus d'une base de données universitaire [MNIST](http://yann.lecun.com/exdb/mnist/)
- Dans un premier temps nous créerons un réseau de neurone simple à un niveau de 10 neurones avec softmax comme fonction d'activation et Cross entropy comme distance pour la descente de gradient.
- Pour la programmation nous utiliserons la bibliothèque tensorflow pour l'apprentissage machine

## Architecture générale du réseau neuronal

On dispose d'une base de données contenant un ensemble de chiffres manuscrits ainsi que leur correspondance (le chiffre exact qu'ils représentent). Cet ensemble sera scindés en deux parties:
- un ensemble d'entraînement (train) qui servira à entraîner notre modèle
- un ensemble de test (test) qui permettra d'évaluer la performance de notre algorithme sur des données qu'il n'a encore jamais vues.

Les données d'entraînement seront fournies en entrée du réseau neuronal à simple couche selon le schéma suivant:

<img src="NN_singleLayer.png"  style="width: 40%; height: 40%"/>

## Equation donnant le vecteur cible Y en fonction de l'entrée X: softmax

<img src="RNN_softmax.png" style="width: 60%; height: 60%"/>

$Y = softmax(X.W + B)$

- X est ici un vecteur de 784 éléments où chacun est la valeur d'un pixel entre 0 et 255 indiquant son niveau de gris. La matrice initiale représentant le chiffre manuscrit M de 28*28 est transformée en un vecteur ou chaque ligne est mise bout à bout: $X_{i+j} = M_{ij}$
- Y est le vecteur cible, là où sont sotckées pour chaque neurone les probabilités d'appartenance à une classe ou à une autre. Ici les classes vont de 0 à 9 et représentent les chiffres que l'on souhaite trouver en fonction de l'image en entrée.
- Y est un vecteur de 10 éléments
- W la matrice des poids (weights) [784,10]
- B le vecteur des biais (biases) [10]

$Y_j = sotfmax( \sum\limits_{i=1}^{784} { X_i.W_{ij}} + B_j)$

Dans chaque neurone est effectuée une transformation du signal d'entrée sous la forme d'une somme des entrées $X_i$ pondérées par les poids $W_{ij}$ à laquelle on ajoute un biais $B_j$ (ce qui permet de donner un degré de liberté supplémentaire au système et déplace la courbe de réponse selon les abscisses vers la droite ou vers la gauche).

L'intérêt de la fonction softmax est qu'elle fait clairement ressortir le plus grand élément tout en gardant le classement des différentes valeurs du vecteur sur lequel on applique la fonction.

Le fonction softmax $\sigma$ est définie de la façon suivante:
$\sigma : R^K \to R^K$

$\sigma(z)_j = \displaystyle\frac{e^{z_j}}{\sum\limits_{k=1}^{K} e^{z_k}}$

$j = 1,...,K$  et $z=(z_1,...,z_K) \in R^K$

En écriture simplifiée on note: $softmax(Z_j) = \displaystyle\frac{e^{Z_j}}{\left\Vert {e^Z}\right\Vert }$

Donc on aura: $$Y_j = \frac{e^{\sum\limits_{i=1}^{784} { X_i.W_{ij}} + B_j}}{\sum\limits_{k=1}^{10} e^{\sum\limits_{i=1}^{784} { X_i.W_{ik}} + B_k}}$$

## Minimisation de la distance: Cross entropy

La seconde étape est de réussir à minimiser la distance entre la solution calculée Y et la solution connue Y'. On pourrait choisir la distance euclidienne mais la littérature nous indique que le meilleur choix pour les classifications est la minimisation de la distance appelée entropie croisée (Cross Entropy).

L'entropie croisée de deux distributions discrètes p et q sur le même espace probabilisé se calcule par:

$H(p,q) = -\sum\limits_{i} p_ilog(q_i)$

Dans notre cas, avec le vecteur Y' qui correspond à la classe connue de la solution. Dans notre exemple pour le 8 : [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]

$-\sum\limits_jY'_jlog(Y_j)$

Entrainer le réseau neuronal correspond à checher des poids de des biais qui minimisent cette fonction de coût (loss function ou cost function).

L'entropie croisée est une fonction de du vecteur d'entrée X (ici les pixels), de la valeur connue de sortie Y' et des poids et des biais.

$$-\sum\limits_{j=1}^{10}Y'_jlog(\frac{e^{\sum\limits_{i=1}^{784} { X_i.W_{ij}} + B_j}}{\sum\limits_{k=1}^{10} e^{\sum\limits_{i=1}^{784} { X_i.W_{ik}} + B_k}})$$

Pour minimiser cette fontion on utilise la méthode de la descente de gradient qui consiste à calculer les dérivées partielles de la fonction de coût selon les poids et les biais. Le gradient pointe vers le haut, pour trouver un minimum local il suffit de partir dans la direction opposée pour chacunes des variables (qui peuvent être très nombreuses, dans notre cas de figure 784 poids $W_{ij}$ et 10 biais $B_j$) et de faire de nouveau les calculs avec les valeurs modifiées d'un incrément donné (suffisament petit pour ne pas dépasser le minimum local).

<img src="cross_entropy.png" style="width: 50%; height: 50%"/>

Dans l'image ci-dessus l'entropie croisée est représentée en fonction de deux poids $W_1$ et $W_2$, et les courbes en noir représentent un exemple du processus d'apprentissage pour obtenir des valeurs de W qui correspondent à un minimum local (on peut noter la sensibilité aux conditions initiales qui font converger vers des minima différents)

# Mini Batch

> Contrairement à la notation mathématique usuelle, les indices en python commencent à 0 pour le premier élément. Que ce soit, un vecteur, une matrice ou un tenseur, pour une dimension données à n éléments les indices varieront de 0 à n-1

Au lieu de faire une descente de gradient stochastique (une entrée à la fois), on utilise un batch d'un certain nombre d'entrées X en même temps. Deux avantages à cela, les contraintes sur le gradient représentent mieux une variété de possibilités puique l'on a plusieurs images en entrée et on travaillera sur de plus grandes matrices. En général sur les GPU, ces calculs matriciels sont plus facile à optimiser.

Par exemple pour un batch de 100 images en entrée on aura X[100,784] on aura la formule suivante:

<img src="softmax_matrix.png" style="width: 50%; height: 50%"/>

Le "broadcasting" pour l'addition en python et numpy correspond à une extension de l'addition usuelle de matrices lorsque les dimensions ne correspondent pas. Ici par exemple la matrice des biais est répétée 100 fois pour que les dimensions concordent.

Prochaine étape: récupération des éléments étiquetés de la bibliothèque d'images et utilisation de tensorflow pour la construction du modèle.